In [ ]:
"visual examples of audio filters"
"Spectrogram visualization"
"Interactive widgets (sliders)"
"Classical DSP + Deep Learning options"

In [8]:
"importing libraries"

import numpy as np
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy import signal
import noisereduce as nr
import pyloudnorm as pyln
from IPython.display import Audio
import ipywidgets as widgets

In [9]:
plt.rcParams['figure.figsize'] = (12, 4)


In [18]:
# ------------------------------
# 🔊 2. Audio I/O Utilities
# ------------------------------

path = "Audio 3_02.wav"

def read_audio(path, target_sr=None, mono=False):
    data, sr = sf.read(path, always_2d=True)
    if mono:
        data = np.mean(data, axis=1, keepdims=True)
    if target_sr and sr != target_sr:   
        data = librosa.resample(data.T, sr, target_sr).T
        sr = target_sr
    return data.astype('float32'), sr


def write_wav_16(path, x, sr):
    q = 2**15 - 1   
    dither = (np.random.rand(*x.shape) - np.random.rand(*x.shape)) / q
    y = np.clip(x + dither, -1, 1)  
    sf.write(path, (y * q).astype(np.int16), sr, subtype='PCM_16')



In [19]:

# ------------------------------
# 📈 3. Spectrogram Visualization
# ------------------------------

def plot_spectrogram(x, sr, title="Spectrogram"):
    if x.ndim > 1:
        x = np.mean(x, axis=1)
    S = librosa.stft(x, n_fft=2048, hop_length=512)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    plt.figure()
    librosa.display.specshow(S_db, sr=sr, hop_length=512, x_axis='time', y_axis='hz')
    plt.colorbar(format="%+2.0f dB")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [21]:
# ------------------------------
# ⚙️ 4. Restoration DSP Blocks
# ------------------------------
def remove_clicks(x, kernel=7):
    from scipy.signal import medfilt
    y = np.zeros_like(x)
    for c in range(x.shape[1]):
        y[:,c] = medfilt(x[:,c], kernel)
    return y


def notch_filter(sig, sr, freq=60, Q=30):
    b, a = signal.iirnotch(freq/(sr/2), Q)
    return signal.filtfilt(b, a, sig)


def remove_hum(x, sr, freqs=(60,120,180)):
    y = x.copy()
    for f in freqs:
        for c in range(y.shape[1]):
            y[:,c] = notch_filter(y[:,c], sr, f)
    return y 


def broadband_denoise(x, sr, noise_seconds=0.5):
    n = int(noise_seconds * sr)
    noise = x[:n]
    y = np.zeros_like(x)
    for c in range(x.shape[1]):
        y[:,c] = nr.reduce_noise(y=x[:,c], sr=sr, y_noise=noise[:,c])
    return y


def highpass(x, sr, cutoff):
    sos = signal.butter(4, cutoff, 'highpass', fs=sr, output='sos')
    y = x.copy()
    for c in range(x.shape[1]):
        y[:,c] = signal.sosfiltfilt(sos, x[:,c])
    return y


def normalize_loudness(x, sr, target=-16):
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(np.mean(x, axis=1))
    y = np.zeros_like(x)
    for c in range(x.shape[1]):
        y[:,c] = pyln.normalize.loudness(x[:,c], loudness, target)
    return y



In [20]:
# ------------------------------
# 🎛️ 5. Interactive Controls
# ------------------------------
def interactive_pipeline(audio, sr):
    hp = widgets.IntSlider(20, 20, 300, 10, description="HPF Hz")
    hum = widgets.Checkbox(True, description="Remove Hum")
    clicks = widgets.Checkbox(True, description="De-click")
    denoise = widgets.Checkbox(True, description="Denoise")

    def process(hp, hum, clicks, denoise):
        y = audio.copy()
        if clicks:
            y = remove_clicks(y)
        if hum:
            y = remove_hum(y, sr)
        if denoise:
            y = broadband_denoise(y, sr)
        y = highpass(y, sr, hp)
        plot_spectrogram(y, sr, "Processed")
        display(Audio(y.T, rate=sr))

    return widgets.interactive(process, hp=hp, hum=hum, clicks=clicks, denoise=denoise)
